In [1]:
from itertools import combinations
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode
import networkx as nx

In [2]:
PATH_TO_DATASETS = 'file:///home/wyvern/ING2/HACKATHON-2025/datasets/'

In [ ]:
spark = SparkSession.builder.appName("MovieSimilarityGraph").getOrCreate()

In [ ]:
titles_df = spark.read.csv(PATH_TO_DATASETS + 'title.basics.tsv', sep='\t', header=True)
ratings_df = spark.read.csv(PATH_TO_DATASETS + 'title.ratings.tsv', sep='\t', header=True)

movies_df = titles_df.join(ratings_df, 'tconst')

In [ ]:
genres_df = movies_df.withColumn("genre", explode(split(movies_df["genres"], ',')))
movies_genres = genres_df.select("tconst", "genre").rdd.collect()

In [ ]:
G = nx.Graph()

for movie1, movie2 in combinations(movies_genres, 2):
    if movie1[1] == movie2[1]:
        G.add_edge(movie1[0], movie2[0])

In [ ]:
nx.write_graphml(G, 'movie_similarity_graph.graphml')

In [ ]:
spark.stop()